# CS440 MP1

In [173]:
import urllib2
from collections import deque
import Queue

## Maze Class and Maze Solver Class

In [205]:
class Maze:
    '''
    Creates a data structure to represent the maze problem.
    '''
    def __init__(self, txtFile):
        '''
        Initializes the maze as a two dimensional list.
        input: text file representing the maze
        printed output: none
        return: none
        '''
        i = 0 #row index
        data = urllib2.urlopen(txtFile)
        self.maze = []
        self.goals = []
        self.explored = []
        for line in data:
            j = 0 #column index
            self.maze.append([])
            for char in line:
                #labels goal state or goal states in maze
                if char == '.':
                    self.goals.append((i,j))
                #labels initial state in maze
                elif char == 'P':
                    self.initialState = (i,j)
                if char == 'P' or char == '.' or char == '%' or char == ' ': 
                    self.maze[i].append(char)
                    j+=1
            i+=1
        self.height = i #set the height
        self.width = j #set the width
        
    def findChildren(self, node):
        '''
        Finds children of a node in maze.
        '''
        children = []
        x = node[1]
        y = node[0]
        if x-1 >= 0:
            if self.maze[y][x-1] != '%':
                children.append((y,x-1))
                
        if y-1 >= 0:
            if self.maze[y-1][x] != '%':
                children.append((y-1,x))
            
        if x+1 < self.width:
            if self.maze[y][x+1] != '%':
                children.append((y,x+1))

        if y+1 < self.height:
            if self.maze[y+1][x] != '%':
                children.append((y+1,x))
        return children
    
    def printMazeWithPath(self, path):
        '''
        prints the maze with the path
        input: sequence of nodes representing the path taken by the algorithm from start to goal.
        printed output: printed maze with path as a sequence of "."
        return: none
        '''
        for row in range(self.height):
            for col in range(self.width):
                if (row,col) in path:
                    print ".",
                else:
                    print self.maze[row][col],
            print " "
        

class MazeSearch:
    '''
    Base class outlining functions that the inherited search classes should contain.
    Should not be created or implemented.
    '''
    def __init__(self, maze):
        '''
        Initializes the maze finding algorithm class. 
        input: A Maze object.
        output: none
        return: none
        '''
        self.frontier #number of nodes that are waiting to be visited
        self.parent = [[(0,0) for j in range(self.maze.width)] for i in range(maze.height)]
        self.maze = maze
        self.numNodesVisited #keeps track of number of nodes visited.
        self.explored
        raise NotImplementedError #added this line so that an object from this class cannot be created.
    
    def pathFinder(self):
        '''
        Finds path in maze from initial state to goal state.
        input: none
        output: none
        return: returns a sequence of nodes in the maze representing a path
        '''
        self.addNodeToFrontier(self.maze.initialState) #adds the initial node to the frontier 
        while not self.emptyFrontier() :#checks if frontier is empty
            self.numNodesVisited+=1
            node = self.chooseNodeFromFrontier()
            
            #check if goal state is reached
            if self.goalState(node):
                return self.backTrace(node)
            
            #put the children of the current state on to the frontier.
            for child in self.maze.findChildren(node):
                if not self.duplicateDetection(child):
                    self.addNodeToFrontier(child)
                    self.parent[child[0]][child[1]] = node
                    self.explored.append(node)
        return -1
    
    def backTrace(self, node):
        '''
        Go back through the list of nodes that were visited and returns the path as a sequence of nodes from 
        goal to initial state.
        input: the goal state node.
        output: none
        return: the path from goal to initial state.
        '''
        path = []
        curNode = node
        
        #check the initial state has not been reached
        while curNode != self.maze.initialState:
            path.append(curNode) #add the current node to the path
            parent = self.parent[curNode[0]][curNode[1]]
            curNode = parent #go from the current node to the parent node.
        return path
    
    def duplicateDetection(self, node):
        '''
        Implement strategy for detecting duplicate states.
        input: node
        printed output: none
        return: true if a duplicate of the node is detected and we do want to not add that node to the 
        frontier. 
        If a duplicate is detected and we want to keep the node on the frontier and remove the existing node, 
        the function returns false and change the frontier.
        '''
        raise NotImplementedError
    
    def addNodeToFrontier(self, node):
        '''
        Adds node to the frontier. The data structure representing a frontier might differ
        for different search strategies.
        input: none
        printed output: none
        return: none
        '''
        raise NotImplementedError
        
    def chooseNodeFromFrontier(self):
        '''
        Implements a strategy for choosing a node from the frontier
        input: none
        printed: none
        return: node chosen by strategy 
        '''
        raise NotImplementedError
        
    def emptyFrontier(self):
        '''
        Checks if the frontier is empty.
        input: none
        printed output: none
        return: true if the frontier is empty
        '''
        raise NotImplementedError
        
    def goalState(self, node):
        '''
        Checks if the goal state has been reached
        '''
        raise NotImplementedError

##Depth First Search 

In [212]:
class DFSMazeSearch(MazeSearch):
    def __init__(self, maze):
        self.frontier = [] #list is used as a stack
        self.parent = [[(0,0) for j in range(maze.width)] for i in range(maze.height)]
        self.maze = maze
        self.numNodesVisited = 0
        self.explored = []
    
    def duplicateDetection(self, node):
        if node in self.explored:
            return True
        
    def addNodeToFrontier(self, node):
        self.frontier.append(node)
        
    def chooseNodeFromFrontier(self):
        return self.frontier.pop()
    
    def emptyFrontier(self):
        return not self.frontier
    
    def goalState(self, node):
        return node in self.maze.goals

In [213]:
medMaze = Maze("http://slazebni.cs.illinois.edu/fall15/assignment1/mediumMaze.txt")
DFS = DFSMazeSearch(medMaze)
path = DFS.pathFinder()
medMaze.printMazeWithPath(path)

% % % % % % % % % % % % % % % % % % % % % % %  
% . %       %       %       %       %   %   %  
% .     %       % %   %   % % % %   % %     %  
% . %       %   %   %   %               %   %  
% . % %   % % %           % %   % %   % % % %  
% . %   %       %   %   %   %   %           %  
% . % % %   %   % % %     %         % %   % %  
% . %   %     . . .     %   %   % . . . %   %  
% . . .     % . % . % % %       % . % .     %  
%   % . %   % .   . . . %   %   % .   . %   %  
%     . % % % . % % % . %   % . . . % . % % %  
%   % . . . . . %   % . . . . . % . . .     %  
% %   %   % %         % % % %   % . % %   % %  
%   %   %       %   %   . . . . . .     %   %  
%       % %   % %   % % . % % % %   % %     %  
%   %   %           % . .       %   %   %   %  
% % % %   % %   % %   . % %   %       %   % %  
%   %   %           % . .   %   %   %   %   %  
%   % %   %   % % % % % . %       . . .     %  
%   %               %   . . %   % . % . %   %  
%       % %   % %   % % % . % . . . % . 

##Breadth First Search

In [214]:
class BFSMazeSearch(MazeSearch):
    def __init__(self, maze):
        self.frontier = deque([])
        self.maze = maze
        self.parent = [[(0,0) for j in range(maze.width)] for i in range(maze.height)]
        self.numNodesVisited = 0
        self.explored = []
    
    def duplicateDetection(self, node):
        if node in self.explored:
            return True
        if node in self.frontier:
            return True
        
    def addNodeToFrontier(self, node):
        self.frontier.append(node)
        
    def chooseNodeFromFrontier(self):
        return self.frontier.popleft()
    
    def emptyFrontier(self):
        return not self.frontier
    
    def goalState(self, node):
        return node in self.maze.goals

In [215]:
medMaze = Maze("http://slazebni.cs.illinois.edu/fall15/assignment1/mediumMaze.txt")
BFS = BFSMazeSearch(medMaze)
path = BFS.pathFinder()
medMaze.printMazeWithPath(path)

% % % % % % % % % % % % % % % % % % % % % % %  
% . %       %       %       %       %   %   %  
% .     %       % %   %   % % % %   % %     %  
% . %       %   %   %   %               %   %  
% . % %   % % %           % %   % %   % % % %  
% . %   %       %   %   %   %   %           %  
% . % % %   %   % % %     %         % %   % %  
% . %   % . . .         %   %   %       %   %  
% . . . . . % . %   % % %       %   %       %  
%   %   %   % . . . . . %   %   %       %   %  
%       % % %   % % % . %   %       %   % % %  
%   %           %   % . . . . . %           %  
% %   %   % %         % % % % . %   % %   % %  
%   %   %       %   %         . . .     %   %  
%       % %   % %   % %   % % % % . % %     %  
%   %   %           %           % . %   %   %  
% % % %   % %   % %     % %   %   .   %   % %  
%   %   %           %       %   % . %   %   %  
%   % %   %   % % % % %   %       . . .     %  
%   %               %       %   %   % . %   %  
%       % %   % %   % % %   %       % . 

In [ ]:
class GBFSMazeSearch(MazeSearch):
    def __init__(self, maze):
        self.frontier = Queue.PriorityQueue()
        self.maze = maze
        self.parent = [[(0,0) for j in range(maze.width)] for i in range(maze.height)]
        self.numNodesVisited = 0
        self.explored = []
    
    def duplicateDetection(self, node):
        if node in self.explored:
            return True
        
    def addNodeToFrontier(self, node):
        self.frontier.put(node)
        
    def chooseNodeFromFrontier(self):
        return self.frontier.get()
    
    def emptyFrontier(self):
        return not self.frontier
    
    def goalState(self, node):
        return node in self.maze.goals